# Structure of standard forward model

The forward model refers to the set of functions in which the electromagnetic spectrum of a planetary atmosphere is calculated based on the information from the reference classes and the selected model parameterisations. In archNEMESIS, all these functions are implemented in the ForwardModel class. In essence, the inputs of the ForwardModel class are the reference classes and the Variables class, and the information within this class is used to perform the radiative transfer calculations and calculate the spectra.

The figure below shows a sketch of the most relevant functions within the standard forward model implementation in archNEMESIS. As mentioned, the inputs of the ForwardModel class are the reference classes and the model parameterisations, which are fed into the subprofretg method of this class. In this function, the reference classes are modified based on the selected model parameterisations, and the updated information is returned. After this, the calc_path method of the ForwardModel class splits the atmosphere into a finite number of layers and the atmospheric paths are calculated based on the geometry of the observation. CIRSrad is the main method where the radiative transfer calculations are performed, first calculating the relevant optical properties of each layer (e.g., absorption and scattering optical depths), solving the multiple scattering field if required (Plass et al., 1973), and finally calculating the electromagnetic spectrum of the planet. After the spectrum has been calculated, the subspecret method is in charge of modifying it based on any relevant model parameterisations that might be added to mimic some instrument characteristics. Finally, the updated spectrum is convolved with the instrument lineshape to account for the spectral resolution of the instrument. 

<img src="images/archNEMESIS_structure_v1.png" alt="Drawing" style="width: 800px;"/>

# Special cases